# Encryprtion/Decryption/Hashing

### What is RSA?
RSA is like having two special keys: one you can share with everyone (public key) and one that only you should have (private key). These keys are used to lock (encrypt) and unlock (decrypt) secret messages.

- Public Key: This is like a padlock you give to anyone. People can use it to lock a message for you.
- Private Key: This is like the key to the padlock. Only you have it, and you use it to unlock the messages sent to you.

### 1. Key Generation (Creating the Keys)
In the code, we're using a function called generate_keys() to make both keys:

- Public key: Used to lock the message.
- Private key: Used to unlock the message.

*public_key, private_key = generate_keys()*

After making the keys, the code converts them into a simple format (hexadecimal, which is like a simplified number format) so we can store or share them easily. That’s what serialize_private_key_to_hex() and serialize_public_key_to_hex() do.

#### --> AFTER YOU GENERATE A RANDOM KEY PAIR, PLEASE KEEP IT SAFE, PUBLIC KEY MUST BE PUBLISHD IN CANVAS, PRIVATE KEY MUST BE KEPT IN SECRET AND SAFETY STORED !!!! 

### 2. Encryption (Locking the Message)

To consider a key secure, key sizes are 2048 bits or longer. In this case, we are using a key size of 2048

Now, someone has your public key and they want to send you a secret message. They use this key to lock (encrypt) the message so no one else can read it. Here's the process in the code:

*encrypted_message = encrypt_message(plaintext_message, public_key)*

The person writes a message (like "Hello").
The message is "locked" using your public key.
The result is a weird-looking scrambled message that only you can unlock.
This step is important because even if someone intercepts the scrambled message, they can’t understand it without the private key.

### 3. Decryption (Unlocking the Message)
When you get the locked message, you use your private key to unlock (decrypt) it and read the original message.

*decrypted_message = decrypt_message(encrypted_message, private_key)*

You take the scrambled message.
Use your private key to unlock it.
Now, you can see the original message.


In [95]:
## Import all the necessary libraries for the code
import base64
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend


#########################################################################
# GENERATE KEYS -> STUDENTS
def generate_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=1024
    )
    public_key = private_key.public_key()
    return public_key, private_key
#########################################################################


#########################################################################
## KEY SERIALIZATION FOR GENERATE KEY PAIR
def serialize_key_to_hex(key):
    if isinstance(key, rsa.RSAPrivateKey):
        key_bytes = key.private_bytes(
            encoding=serialization.Encoding.DER,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        )
    elif isinstance(key, rsa.RSAPublicKey):
        key_bytes = key.public_bytes(
            encoding=serialization.Encoding.DER,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
    else:
        raise ValueError("Invalid key type")
    
    return key_bytes.hex()
#########################################################################

    
#########################################################################
# ENCRYPTION
def encrypt_message_with_public_key(plaintext_message, pb_key_hex):
    """
    Encrypts a message using a public key in hex format.
    
    :param plaintext_message: The message to encrypt
    :param pb_key_hex: The public key in hex format
    :return: The encrypted message in base64 format
    """
    # Step 1: Deserialize the public key from hex
    public_key_bytes = bytes.fromhex(pb_key_hex)
    public_key = serialization.load_der_public_key(public_key_bytes)

    # Step 2: Encrypt the message using the public key
    ciphertext = public_key.encrypt(
        plaintext_message.encode('utf-8'),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    # Step 3: Return the encrypted message in base64 format
    return base64.b64encode(ciphertext).decode('utf-8')
#########################################################################



#########################################################################
# DECRYPTION
def decrypt_message_with_private_key(encrypted_message, pr_key_hex):
    """
    Decrypts a message using a private key in hex format.
    
    :param encrypted_message: The encrypted message (base64 encoded ciphertext)
    :param pr_key_hex: The private key in hex format
    :return: The decrypted message (plaintext)
    """
    # Step 1: Deserialize the private key from hex
    private_key_bytes = bytes.fromhex(pr_key_hex)
    private_key = serialization.load_der_private_key(
        private_key_bytes,
        password=None
    )

    # Step 2: Decrypt the message using the private key
    decrypted = private_key.decrypt(
        base64.b64decode(encrypted_message),  # Decode from base64 to bytes
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    # Step 3: Return the decrypted message as a string
    return decrypted.decode('utf-8')
#########################################################################


#########################################################################
# DIGITAL SIGNATURE
def sign_message_with_private_key(message, pr_key_hex):
    """
    Signs a message using a private key in hex format.
    
    :param message: The message to sign
    :param pr_key_hex: The private key in hex format
    :return: The digital signature in base64 format
    """
    # Step 1: Deserialize the private key from hex
    private_key_bytes = bytes.fromhex(pr_key_hex)
    private_key = serialization.load_der_private_key(
        private_key_bytes,
        password=None
    )

    # Step 2: Sign the message using the private key
    signature = private_key.sign(
        message.encode('utf-8'),  # Convert the message to bytes
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    
    # Step 3: Return the signature in base64 format
    return base64.b64encode(signature).decode('utf-8')
#########################################################################

#########################################################################
# VERIFY DIGITAL SIGNATURE
def verify_signature_with_public_key(message, signature, pb_key_hex):
    """
    Verifies a digital signature using a public key in hex format.
    
    :param message: The original message that was signed
    :param signature: The Base64-encoded signature to verify
    :param pb_key_hex: The public key in hex format
    :return: True if the signature is valid, False otherwise
    """
    try:
        # Step 1: Deserialize the public key from hex
        public_key_bytes = bytes.fromhex(pb_key_hex)
        public_key = serialization.load_der_public_key(public_key_bytes)

        # Step 2: Verify the signature
        public_key.verify(
            base64.b64decode(signature),  # Decode Base64 signature to bytes
            message.encode('utf-8'),  # Convert message to bytes
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        return False

#########################################################################
# Hashing function
def get_text_hash(text: str) -> str:
    # Create a SHA-256 hash object
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    
    # Update the hash object with the bytes of the text
    digest.update(text.encode('utf-8'))
    
    # Finalize the hash and get the digest
    hash_bytes = digest.finalize()
    
    # Convert the hash bytes to a hexadecimal string
    return hash_bytes.hex()




In [88]:
# Block 1: Key Generation

public_key, private_key = generate_keys()
print("Keys generated successfully. \n")
print(f"Private key (hex format): \n{serialize_key_to_hex(private_key)} \n")
print(f"Public key (hex format): \n{serialize_key_to_hex(public_key)} \n")

Keys generated successfully. 

Private key (hex format): 
30820139020100024100cca92d34af134930ce25afe683d5d3ac120a7342c58cbf063ed471569359f28a8dbadf6883e784b8ad873dbede582ec5a24a62f9c2a80463b893d9b94f70f1fd0203010001024057879f52e1f1d0a2b3f80054095c612c3d8704a6b110b1fe3ccee52e4ed4f05ab7c23b1086d20aee558e8284d11bf5bb851138881c37cd0928cf48ec45f6ae8d022100ec19375f4c08bcb292a19d46d3d0e84da9c7b1d1f767e83613ef8ef8481dcd23022100dde990b3d019bb060963c2240918b53293b77a42bd60ab16c3f017af20d1065f02200a5e391e483a81fedd8d97588ae8bbaadd6fd59140ea213a6eb9b06f9acb243d022025359dc2d8d907a81a78f4bbb01ec97b429398578a83a40c7c2c4cfc5b87510702205a1a80170c87b4c13f7904b07a87ea14c711d7bb4fa4fa4f49383ccc3122674f 

Public key (hex format): 
305c300d06092a864886f70d0101010500034b003048024100cca92d34af134930ce25afe683d5d3ac120a7342c58cbf063ed471569359f28a8dbadf6883e784b8ad873dbede582ec5a24a62f9c2a80463b893d9b94f70f1fd0203010001 



## ENCRYPTION/DECRYPTION


In [22]:
# ENCRYPTION
# Input message and public key
plaintext_message = input('Enter the message to encrypt: ')
pb_key_hex = input('Enter the public key (hex format) for encryption: ')

# Encrypt the message and print the result
encrypted_message = encrypt_message_with_public_key(plaintext_message, pb_key_hex)
print(f"\nEncrypted message: {encrypted_message} \n")

Enter the message to encrypt:  123456
Enter the public key (hex format) for encryption:  30820122300d06092a864886f70d01010105000382010f003082010a0282010100b8ad61ea97de686ecf9ea7317cced9a53d6d49bc1caf9b5cf439e58fa5c725574c84872dce79b8d9ad17aca8ac466e470e91207185901f61efa9223dc46e49d19e76824da4f88b06b7b9d6eab0f741d71e63c7cafce8b21684b710cc33e6a45a2f17e4f7ce9fdf4a1841eadebf085caf7980ed869067cd10c996e69a7156c46fc49772cba6ab87c80c2dd4d99448c1675f08fb84c83b9911f85cea0efa5b27a27d46711ed96a58a33d8b8013d4ca90d59b5e338ef29bdde9eaa1823cbfe6dd5fd11eb256b8469ae773e16e0d2a8a4685096ad973226f75809dbc30cc7b5f69a5d23419013bb9d932c6760bb1a43ea17cf59f9dd0bae2ff86ab7bc080747aca3d0203010001



Encrypted message: rAjTdnUBB/DyX4OxWQWKDfTYTsVJV5uAdCcALDsiiff1d4Y29je5ZuaQX7MKf8qVCzsHuVQ4alOLG+vyTUPNZrN9LMabxGWHTxiQmztcub/w+8MH46UkvYOPOstNcUxBdkWmq1PmINGB0P5qB5u/cqvoLVbz85HQv26h5pFHrWZTu3xbx5ZLLsILvIPfRxMV9DCVVyYC8GKIha+xUZ+LicNvYxuZWP23OT1N4b5aqvgNLZ7rTS3D2WSofYcfH+Cmbr/cCwfnTL7+W28C0INp70wqaEK5qknOAMDhnI89M1Eu4/aTaFkraTu6SXwZtcqYcxJcx7ozY9atzK7FI9J20w== 



In [19]:
# DECRYPTION
# Input encrypted message and private key
encrypted_message = input('Enter the encrypted message (ciphertext): ')
pr_key_hex = input('Enter the private key (hex format) for decryption: ')

# Decrypt the message and print the result
decrypted_message = decrypt_message_with_private_key(encrypted_message, pr_key_hex)
print(f"\nDecrypted message: {decrypted_message} \n")

Enter the encrypted message (ciphertext):  P6ic9UayQ9JWpPCXWFOUOYZY1WPtcZF3xLl38h6bJYBQitDqdxVI8VVi8/EkHfWImq0HEXRksO+GqJw2zAgIncmUaOmNrDNcieEnTug4ZYH9yIxKXBOlE0Rnrr/YAbg4NasT72uGr7VIqGQmMS+ko7yB4XkiCYjmnjawrHh/Eh1Cj4+RLR8SCHuQEJq2CldAtQxqdLb0AmafG0HhWVHw0H6bjz84POTcrJbovPjgp8XyAAOtIiNHxX9knb3VAVE5SkzYhL1+SBuw9V1ohBEyCVrFBimZLn01Pqd27VJ8skk+ErVE6qUdyah16sQK6rOBL9UdBzOZuMEK9c46J/JCeQ==
Enter the private key (hex format) for decryption:  308204a30201000282010100c18968700a5d93d24a48d77ff9bfafc2ab3df0e24342a9bbfff07b2c8bc46ba5c22751313288663ef2de5a4c933acadc039c2b68062d00651849c4a6d2bc3a7e919fad252f22c1fc0bca27583844f96b43dc89cc2b647a6bc8c09997f08b20979e1ba812988665c402b07fddd943298e662c9bc32a01192ced69a29d34b06fe26db9c672509a731971fe6ec90d972081df50a4fe051ebc38aaa75650cc176ebe62c3f5ff88eff1b4ea68794e1b888e1ace7665f4baf50b3b69128d42e0cb220e2206ed72e0a10bf8fe3dd4e6128e0c888d96a54a9625a4dc579d0065714aad8ca9bcc3e704855bc38c1e8ff392bdeb43aec4df7c6d04fda350b009be7f2a400f0203010001028201000dbda0


Decrypted message: Hola mundo 



## What is a Digital Signature? (Non-repudiation)
A digital signature is like signing a letter with your unique signature, but instead of using a pen, you use your private key to "sign" the message. When someone else gets your signed message, they can use your public key to check if the message is really from you and if it hasn't been tampered with.

Here’s the process:

Sign the Message: You "sign" the message using your private key.
Verify the Signature: Anyone can check the signature with your public key to make sure it's really you who sent the message and that it hasn’t been altered.

### Digital Signature (Signing the Message)
This is where you take a message and create a digital signature with your private key. It’s like putting your personal stamp on the message so others know it’s truly from you.

*signature = sign_message(message_to_sign, private_key)*

You create a signature using your private key. This signature is unique to both your message and your private key.
The signature is like a scrambled code that only your private key could create.
The message itself is not encrypted here, but it’s signed so that people can verify its authenticity.

#### Why Sign a Message?
- Proof of Identity: It proves the message came from you (the private key holder).
- Integrity: It ensures the message wasn’t altered during transmission.

### Signature Verification (Checking the Signature)
Now, if someone receives your signed message, they can check that it's really from you by verifying it with your public key.

In the code:

*is_valid = verify_signature(message_to_sign, signature_to_verify, public_key)*

Here’s what happens:

Verification: The public key is used to check the signature and confirm if it matches the message. If it does, the signature is valid.
If the message or signature doesn’t match, it means something is wrong (like someone tried to change the message), and the verification will fail.

Simple Example:
1. Alice writes a message and signs it with her private key.
2. Bob receives the message and Alice’s signature.
3. Bob uses Alice’s public key to verify that:
4. The message really came from Alice.
5. The message hasn’t been changed.


### What is Necessary for Signature Verification:
- Original Message: The message that was signed.
- Digital Signature: A unique signature created by the sender using their private key.
- Public Key: The public key of the sender used to verify the signature.
- Hashing Algorithm: A function that converts the message into a fixed-length string (e.g., SHA-256).
- Signing Algorithm: RSA (with padding) used to sign the hashed message.

Process:
1. Hash the Message: The recipient hashes the original message using the same algorithm (e.g., SHA-256).
2. Decrypt the Signature: The signature is decrypted using the sender’s public key to retrieve the original hash.
3. Compare Hashes: If the decrypted hash matches the hash of the message, the signature is valid. Otherwise, it’s invalid.


In [97]:
# DIGITAL SIGNATURE

# Input message and private key for signing
message_to_sign = input('Enter the message to digitally sign: ')
pr_key_hex = input('Enter your private key (hex format) for signing: ')

# Sign the message and print the result
signature = sign_message_with_private_key(message_to_sign, pr_key_hex)
print(f"\nDigital signature: {signature} \n")


Enter the message to digitally sign:  This is my message
Enter your private key (hex format) for signing:  308204a20201000282010100b8ad61ea97de686ecf9ea7317cced9a53d6d49bc1caf9b5cf439e58fa5c725574c84872dce79b8d9ad17aca8ac466e470e91207185901f61efa9223dc46e49d19e76824da4f88b06b7b9d6eab0f741d71e63c7cafce8b21684b710cc33e6a45a2f17e4f7ce9fdf4a1841eadebf085caf7980ed869067cd10c996e69a7156c46fc49772cba6ab87c80c2dd4d99448c1675f08fb84c83b9911f85cea0efa5b27a27d46711ed96a58a33d8b8013d4ca90d59b5e338ef29bdde9eaa1823cbfe6dd5fd11eb256b8469ae773e16e0d2a8a4685096ad973226f75809dbc30cc7b5f69a5d23419013bb9d932c6760bb1a43ea17cf59f9dd0bae2ff86ab7bc080747aca3d02030100010282010029a2a346efb6cb7e220e4f7ce379c20d542839bdb08cf1beb6ebffa28f5a32c3b8034ff3a68f803a8920e3830295fd6245393862879d6ac9da6d9c03e76f82f280a290b57f1a2ec3d4b38fedc8f871ae555263169346c5f1b8398a8eb32bce456bc51c10b75389548bdbfe9ef9dc218349262546d773238323a3c2f93250c0c7dce5d0c9e314fc15a4e26a5422bc4694c43538febb98b236411b7a64a63d9cc51558b8f4253606eb08e


Digital signature: dGYQ1ppMHnHZffDUcNn8KJLYxInw2wnuhO7lk0gO/TDfYKdr3iMobNRyEyHQIFBO5qTMg9pPjlSFoH28t4pndsamgJRTRkkMZ2TUxK0t0a5c4n2upVfRIStL1EBanlfllPF64M9w1m8WC6oKNuwD4uAdRz6y+ifcUCi4IYu1pt8X65vE5nVn0qMEHNSUufXFogzSV3ZUmMSyzCk+SCz2SoWcW02/b1Zx//1ONx7vFfCWIRBZl9YxsmDNSnoiPSr4DeojkDQ8G8SUzqASAF7Fx4nXgRpDDn3HITXRR43TZkLWpcEORE2697Esqs8W5HseblfVXjGNzlwgBZYzSBFxnA== 



In [99]:
# SIGNATURE VERIFICATION

# Input signature and public key for verification
signature_to_verify = input('Enter the signature to verify: ')
pb_key_hex = input('Enter the public key (hex format) for verification: ')
message_to_sign = input('Enter the message you want to verify: ')

# Verify the signature and print the result
is_valid = verify_signature_with_public_key(message_to_sign, signature_to_verify, pb_key_hex)

if is_valid:
    print("\nThe signature is valid.\n")
    print(f"Original message: {message_to_sign}\n")
else:
    print("\nThe signature is invalid.\n")

Enter the signature to verify:  dGYQ1ppMHnHZffDUcNn8KJLYxInw2wnuhO7lk0gO/TDfYKdr3iMobNRyEyHQIFBO5qTMg9pPjlSFoH28t4pndsamgJRTRkkMZ2TUxK0t0a5c4n2upVfRIStL1EBanlfllPF64M9w1m8WC6oKNuwD4uAdRz6y+ifcUCi4IYu1pt8X65vE5nVn0qMEHNSUufXFogzSV3ZUmMSyzCk+SCz2SoWcW02/b1Zx//1ONx7vFfCWIRBZl9YxsmDNSnoiPSr4DeojkDQ8G8SUzqASAF7Fx4nXgRpDDn3HITXRR43TZkLWpcEORE2697Esqs8W5HseblfVXjGNzlwgBZYzSBFxnA==
Enter the public key (hex format) for verification:  30820122300d06092a864886f70d01010105000382010f003082010a0282010100b8ad61ea97de686ecf9ea7317cced9a53d6d49bc1caf9b5cf439e58fa5c725574c84872dce79b8d9ad17aca8ac466e470e91207185901f61efa9223dc46e49d19e76824da4f88b06b7b9d6eab0f741d71e63c7cafce8b21684b710cc33e6a45a2f17e4f7ce9fdf4a1841eadebf085caf7980ed869067cd10c996e69a7156c46fc49772cba6ab87c80c2dd4d99448c1675f08fb84c83b9911f85cea0efa5b27a27d46711ed96a58a33d8b8013d4ca90d59b5e338ef29bdde9eaa1823cbfe6dd5fd11eb256b8469ae773e16e0d2a8a4685096ad973226f75809dbc30cc7b5f69a5d23419013bb9d932c6760bb1a43ea17cf59f9dd0bae2ff86ab7bc080


The signature is valid.

Original message: This is my message



In [47]:
# HASHING FUNCTION

# Example usage
text = input('Enter the Student ID to get the HASH using SHA256: ')
hash_result = str(get_text_hash(text))
print(f"Hash of the text: {hash_result}")


Enter the Student ID to get the HASH using SHA256:  098765


Hash of the text: 4a9ca4596692e94f9d2912b06a0d007564a22ee750339a6021c2392149b25d6d
